# Autonomous Trader with an Evaluator

We use an evaluator agent which we created as an MCP server in this implementation

In [ ]:
import sys
from pathlib import Path

# For access to resources in the week's home directory
project_root = Path.cwd().parent.parent
sys.path.insert(0, str(project_root))

In [ ]:
import os
from dotenv import load_dotenv
from agents import Agent, Runner, trace, Tool
from agents.mcp import MCPServerStdio
from IPython.display import Markdown, display
from datetime import datetime
from accounts_client import read_accounts_resource, read_strategy_resource
from accounts import Account

load_dotenv(override=True)

In [ ]:
polygon_api_key = os.getenv("POLYGON_API_KEY")
polygon_plan = os.getenv("POLYGON_PLAN")

is_paid_polygon = polygon_plan == "paid"
is_realtime_polygon = polygon_plan == "realtime"


In [ ]:
if is_paid_polygon or is_realtime_polygon:
    market_mcp = {"command": "uvx","args": ["--from", "git+https://github.com/polygon-io/mcp_polygon@master", "mcp_polygon"], "env": {"POLYGON_API_KEY": polygon_api_key}}
else:
    market_mcp = ({"command": "uv", "args": ["run", "market_server.py"]})

trader_mcp_server_params = [
    {"command": "uv", "args": ["run", "accounts_server.py"]},
    {"command": "uv", "args": ["run", "push_server.py"]},
    market_mcp
]

### Researcher, Trader and Evaluator MCP Servers

In [ ]:
brave_env = {"BRAVE_API_KEY": os.getenv("BRAVE_API_KEY")}

researcher_mcp_server_params = [
    {"command": "uvx", "args": ["mcp-server-fetch"]},
    {"command": "npx", "args": ["-y", "@modelcontextprotocol/server-brave-search"], "env": brave_env}
]

In [ ]:

evaluator_server_path = Path.cwd() / "community_contributions" / "kwabena" / "evaluator_server.py"

evaluator_mcp_server_params = [
    {"command": "uv", "args": ["run", str(evaluator_server_path)]}
]

In [ ]:
researcher_mcp_servers = [MCPServerStdio(params, client_session_timeout_seconds=30) for params in researcher_mcp_server_params]
trader_mcp_servers = [MCPServerStdio(params, client_session_timeout_seconds=30) for params in trader_mcp_server_params]
evaluator_mcp_servers = [MCPServerStdio(params, client_session_timeout_seconds=30) for params in evaluator_mcp_server_params]
mcp_servers = trader_mcp_servers + researcher_mcp_servers + evaluator_mcp_servers

## Researcher Agent

In [ ]:
async def get_researcher(mcp_servers) -> Agent:
    instructions = f"""You are a financial researcher. You are able to search the web for interesting financial news,
                    look for possible trading opportunities, and help with research.
                    Based on the request, you carry out necessary research and respond with your findings.
                    Take time to make multiple searches to get a comprehensive overview, and then summarize your findings.
                    If there isn't a specific request, then just respond with investment opportunities based on searching latest news.
                    The current datetime is {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}
                    """
    researcher = Agent(
        name="Researcher",
        instructions=instructions,
        model="gpt-4.1-mini",
        mcp_servers=mcp_servers,
    )
    return researcher

In [ ]:
async def get_researcher_tool(mcp_servers) -> Tool:
    researcher = await get_researcher(mcp_servers)
    return researcher.as_tool(
            tool_name="Researcher",
            tool_description="This tool researches online for news and opportunities, \
                either based on your specific request to look into a certain stock, \
                or generally for notable financial news and opportunities. \
                Describe what kind of research you're looking for."
        )

In [ ]:
research_question = "What's the latest news on Amazon?"

for server in researcher_mcp_servers:
    await server.connect()
researcher = await get_researcher(researcher_mcp_servers)
with trace("Researcher"):
    result = await Runner.run(researcher, research_question, max_turns=30)
display(Markdown(result.final_output))

In [ ]:
project_root = Path.cwd().parent.parent
os.chdir(project_root)
sys.path.insert(0, str(project_root))

In [ ]:
ed_initial_strategy = "You are a day trader that aggressively buys and sells shares based on news and market conditions."
Account.get("Ed").reset(ed_initial_strategy)

display(Markdown(await read_accounts_resource("Ed")))
display(Markdown(await read_strategy_resource("Ed")))

## Trader Agent

In [ ]:
agent_name = "Ed"

# Using MCP Servers to read resources
account_details = await read_accounts_resource(agent_name)
strategy = await read_strategy_resource(agent_name)

instructions = f"""
You are a trader that manages a portfolio of shares. Your name is {agent_name} and your account is under your name, {agent_name}.
You have access to tools that allow you to search the internet for company news, check stock prices, and buy and sell shares.
Your investment strategy for your portfolio is:
{strategy}
Your current holdings and balance is:
{account_details}
You have the tools to perform a websearch for relevant news and information.
You have tools to check stock prices.
You have tools to buy and sell shares.
You have tools to save memory of companies, research and thinking so far.
Please make use of these tools to manage your portfolio. Carry out trades as you see fit; do not wait for instructions or ask for confirmation.
"""

prompt = """
Use your tools to make decisions about your portfolio.
Investigate the news and the market, make your decision, make the trades, and respond with a summary of your actions.
"""

In [ ]:
for server in mcp_servers:
    await server.connect()

researcher_tool = await get_researcher_tool(researcher_mcp_servers)
trader = Agent(
    name=agent_name,
    instructions=instructions,
    tools=[researcher_tool],
    mcp_servers=trader_mcp_servers,
    model="gpt-4o-mini",
)
with trace(agent_name):
    result = await Runner.run(trader, prompt, max_turns=30)
display(Markdown(result.final_output))

### Evaluator Agent

In [ ]:
async def get_evaluator(evaluator_servers, trader_name: str) -> Agent:
    """Creates a simple evaluator agent"""
    
    strategy = await read_strategy_resource(trader_name)
    
    instructions = f"""You evaluate {trader_name}'s trading performance.

Strategy: {strategy}

You have ONE tool: evaluate_performance - gives all key metrics.

Call it once, then provide:
- Brief performance summary
- 2-3 quick recommendations
"""
    
    evaluator = Agent(
        name="Evaluator",
        instructions=instructions,
        model="gpt-4o-mini",
        mcp_servers=evaluator_servers,
    )
    return evaluator

In [ ]:
# Connect evaluator servers
for server in evaluator_mcp_servers:
    await server.connect()

# Create and run evaluator
evaluator = await get_evaluator(evaluator_mcp_servers, agent_name)

with trace("Evaluator"):
    eval_result = await Runner.run(evaluator, "Evaluate the trader's performance.", max_turns=4)

display(Markdown("## Performance Evaluation"))
display(Markdown(eval_result.final_output))